In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from scipy.optimize import minimize

def fetch_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data['Adj Close']

def calculate_daily_returns(stock_data):
    return stock_data.pct_change().dropna()

def calculate_annual_returns(daily_returns):
    return (1 + daily_returns.mean()) ** 252 - 1

def calculate_volatility(daily_returns):
    return daily_returns.std() * np.sqrt(252)

def calculate_sharpe_ratio(returns, volatility):
    return returns / volatility

def objective_function(weights, returns, cov_matrix):
    portfolio_return = np.dot(weights, returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return -portfolio_return / portfolio_volatility  # Negative Sharpe ratio for maximization

def optimize_portfolio(returns, cov_matrix):
    num_assets = len(returns)
    initial_weights = np.ones(num_assets) / num_assets  # Equal weights to start with

    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})  # Sum of weights equals 1

    result = minimize(objective_function, initial_weights, args=(returns, cov_matrix),
                      method='SLSQP', constraints=constraints)

    return result.x

if __name__ == "__main__":
    # Example: Optimizing a portfolio of two stocks (AAPL and TSLA)
    tickers = ['HCL', 'INFY']
    start_date = datetime.now() - timedelta(days=365)
    end_date = datetime.now()

    stock_data = pd.concat([fetch_stock_data(ticker, start_date, end_date) for ticker in tickers], axis=1)
    stock_data.columns = tickers

    daily_returns = calculate_daily_returns(stock_data)
    cov_matrix = daily_returns.cov().to_numpy()

    annual_returns = calculate_annual_returns(daily_returns)
    annual_volatility = calculate_volatility(daily_returns)

    optimal_weights = optimize_portfolio(annual_returns, cov_matrix)

    print("Optimal Weights:", optimal_weights)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
Optimal Weights: [0.92756492 0.07243508]
